In [1]:
### THA STANDARTIZE PLAYGROUND RESEARCH MODE

In [2]:
### MODULES IMPORT
import pandas as pd
import numpy as np
from datetime import date, datetime
import math

In [18]:
### INTERNAL PARAMETERS INITIALIZATION

### Universe path:
str_path_universe = 'Data_Files/Source_Files/acadian_universe.xlsx'
### Bloomberg structured data extraction keys:
str_path_bb_hdf = 'Data_Files/Source_Files/Bloomberg_prepared.h5'
str_key_reer_sourced = 'bb_reer_sourced'
### General daily-mone ranges parameters:
str_source_date_start = '1992-01-01' ### Start date for source vectors
str_measure_date_start = '1996-08-01' ### Start date for efficacy measures
str_measure_date_end = '2020-08-31' ### End date for efficacy measures
idx_source_date_range = pd.date_range(str_source_date_start, str_measure_date_end, freq = 'B') ### Range for source data filtering
idx_factor_date_range = pd.date_range(str_source_date_start, str_measure_date_end, freq = 'BM') ### Range for factor data filtering
idx_measure_date_range = pd.date_range(str_measure_date_start, str_measure_date_end, freq = 'BM') ### Range for measures calculation
### Results saving:
str_reer_tha_xlsx = 'Data_Files/Test_Files/research_mode_tha_reer_std.xlsx'
str_reer_raw_xlsx = 'Data_Files/Test_Files/research_mode_tha_reer_raw.xlsx'

In [4]:
### GENERAL PARAMETERS INITIALIZATION

### Common constants:
All = slice(None)

### Standartization parameters:
list_truncate = [2.5, 2.0] ### Standartization boundaries
bool_within_market = True ### Standartization option
flo_tha_ratio = 0.9 ### THA progression ratio
int_tha_length = 24 ### THA horizon length

### ISON filtering options:
list_ison = ['DM', 'EM', 'FM'] ### Regions filter to drop NaN region values
list_filter = ['DM', 'EM', 'FM'] ### Additional regions filter
list_countries_to_exclude = ['VE'] ### Countries not to play the game

In [5]:
### DEFINING EXPONENTIAL WEIGHT

def exp_weight_single(halflife_len = 3, num_element = 0):
    ### Weight calculating:
    num_period_factor = math.exp(math.log(0.5) / round(halflife_len))
    num_weight = np.exp(math.log(num_period_factor) * num_element)
    ### Result output:
    return num_weight

In [6]:
### DEFINING GEOMETRICAL WEIGHT

def geom_weight_single(flo_ratio, flo_factor = 1, num_element = 0):
    ### Results output:
    return flo_factor * (flo_ratio ** num_element)

In [7]:
### DEFINING WEIGHTED AVERAGE

def weighted_average(ser_data, ser_weight = False, int_min_count = 0):
    ### Default output:
    num_result = np.NaN
    ### Checking for data presence:
    if (ser_data.count() > int_min_count):       
        ### Checking for weights dataset:
        if isinstance(ser_weight, bool):
            ### Calculating of simple average:
            num_result = np.nanmean(ser_data.values)
        else:
            ### Weights filtering:
            list_weight = ser_weight[ser_data.dropna().index].values
            ### Checking for weights presence:
            if np.nansum(list_weight):
                ### Data filtering:
                list_data = ser_data.dropna().values
                ### Weighted average calculating:
                num_result = np.nansum(list_data * list_weight) / np.nansum(list_weight)
    ### Results output:
    return num_result

In [8]:
### DEFINING MEAN MOMENTUM FUNCTION

def rolling_cond_weighted_mean(ser_country_matrix, ser_full_source, int_mean_win, int_mean_min, list_weight = False, ser_full_cond = False):
    ### Defining conditional average calculator:
    def conditional_average(ser_source, list_weight, int_min_count = 0, ser_condition = False):
        ### Weight setting
        ser_weight = pd.Series(list_weight[ : len(ser_source.index)], ser_source.index)
        ### If we have condition we should resort the weight array:
        if not isinstance(ser_condition, bool):
            ser_condition_sorted = pd.Series(ser_condition.sort_values().index, ser_condition.index)
            ser_condition_sorted.name = 'Condition'
            ser_weight = pd.concat([ser_weight, ser_condition_sorted], axis = 1).reset_index(drop = True).set_index('Condition').squeeze().sort_index()            
        ### Results output:
        return weighted_average(ser_source, ser_weight, int_min_count)    
    ### Country saving:
    str_country = ser_country_matrix.index[0][1]
    ### Checking for country presence in source vector:
    if (str_country in ser_full_source.index.get_level_values(1)):
        ### Filtering country vector from source:
        ser_country_source = ser_full_source.loc[All, str_country]
        if not isinstance(ser_full_cond, bool):
            ser_country_cond = ser_full_cond.loc[All, str_country]
        ### Looping over matrix index dates:
        for iter_bm_date in ser_country_matrix.index.get_level_values(0):
            try:
                ### Defining monthend date number in source country vector:
                int_idx_num = ser_country_source.index.get_loc(iter_bm_date)
                ### Creating vectors for numerator and denominator means calculation:
                ser_rolled_source = ser_country_source.iloc[max((int_idx_num - int_mean_win + 1), 0) : int_idx_num + 1]
                if not isinstance(ser_full_cond, bool):
                    ser_rolled_cond = ser_country_cond.loc[ser_rolled_source.index]
                else:
                    ser_rolled_cond = False
                ### Action for MatLab compatibility:
                ser_rolled_source.iloc[0] = np.NaN
                ### Simple mean calculation:
                if isinstance(list_weight, bool):
                    ser_country_matrix.loc[iter_bm_date, str_country] = weighted_average(ser_rolled_source, False, int_mean_min)
                else:
                    ### Weighted mean calculation:
                    ser_country_matrix.loc[iter_bm_date, str_country] = conditional_average(ser_rolled_source, list_weight, int_mean_min, ser_rolled_cond)
            except KeyError:
                pass
    ### Resulting vector output:
    return ser_country_matrix

In [9]:
### DEFINING MULTI-STEP STANDARTIZATION FOR SEPARATE SERIES

def multistep_standartize(ser_data_source, arr_truncate, ser_weight = False, reuse_outliers = False, center_result = True, full_result = False):  
    ### Arrays of iterations properties:
    arr_mean = []
    arr_std = []
    ### Adding equal weights, when weights are absent:
    if isinstance(ser_weight, bool):
        ser_weight = pd.Series(1, index = ser_data_source.index)
        ser_weight.name = 'Weight'    
    ### Workhorse and resulting data vectors initialising:
    ser_data_iter = ser_data_source.dropna()
    ser_weight_iter = ser_weight.copy()
    ser_data_full = pd.Series(np.NaN, index = ser_data_iter.index)
    ### Looping by boundaries array:
    for num_bound_iter in arr_truncate:
        ### Properties calculating and saving:
        num_mean_iter = weighted_average(ser_data_iter, ser_weight_iter)
        num_std_iter = ser_data_iter.std()
        arr_mean.append(num_mean_iter)
        arr_std.append(num_std_iter)
        ser_data_iter = (ser_data_iter - num_mean_iter) / num_std_iter       
        ### Standartizing:
        if reuse_outliers:
            ser_data_iter[ser_data_iter.abs() >= num_bound_iter] = np.sign(ser_data_iter) * num_bound_iter 
        else:
            ### Saving to result and excluding from further calculations truncated values:             
            ser_data_full.where(ser_data_iter.abs() < num_bound_iter, np.sign(ser_data_iter) * num_bound_iter, inplace = True)
            ser_data_iter = ser_data_iter[ser_data_iter.abs() < num_bound_iter]           
    ### Aggregating result:
    if (reuse_outliers):
        ser_data_full = ser_data_iter
    else:     
        ser_data_full[ser_data_iter.index] = ser_data_iter
    ### Centering result:
    if (center_result):
        ser_result = ser_data_full - weighted_average(ser_data_full, ser_weight) 
    else:
        ser_result = ser_data_full    
    ### Result output:
    ser_result.name = str(ser_data_source.name) + '_standartized'
    if (full_result):
        return (ser_result, arr_mean, arr_std)
    else:
        return ser_result

In [10]:
### DEFINING MULTI-STEP STANDARTIZATION BY MARKET FOR CROSS-SECTION

def ison_standartize(ser_to_manage, arr_truncate, ser_weight = False, reuse_outliers = False, center_result = True, full_result = False, within_market = False):
    ### Multi-step standartizing:
    if (within_market):
    ### Within market standartizing:
        ser_result = ser_to_manage.groupby(by = 'Market', group_keys = False).apply(multistep_standartize, arr_truncate, ser_weight, 
                                                                                                  reuse_outliers, center_result, full_result)
    else:
    ### Full universe standartizing:
        ser_result = multistep_standartize(ser_to_manage, arr_truncate, ser_weight, reuse_outliers, center_result, full_result)
    ### Results output:
    return ser_result

In [11]:
### DEFINING MULTI-STEP THA STANDARTIZATION BY MARKET FOR CROSS-SECTION

def tha_standartize(ser_to_manage, arr_truncate, ser_weight = False, reuse_outliers = False, center_result = True, full_result = False):
    ### Multi-step standartizing and recovering data (just for outliers winsorizing only):
    (ser_reversed, list_mean, list_std) = multistep_standartize(ser_to_manage, arr_truncate, ser_weight, reuse_outliers, center_result, full_result = True)
    for iter_num in range(len(arr_truncate))[::-1]:
        ser_reversed = (ser_reversed * list_std[iter_num] + list_mean[iter_num])
    ### By market demeaning for winsorized data vector:
    ser_demeaned = ser_reversed.groupby('Market').apply(lambda ser_region: ser_region - ser_region.mean())
    ### Winsorized & demeaned by market data vector standartizing:
    ser_stand_z = multistep_standartize(ser_demeaned, arr_truncate, ser_weight, reuse_outliers, center_result, full_result = False)
    ### Results output:
    return ser_stand_z

In [12]:
### DEFINING UNIVERSAL AUTOCORRELATION FOR DATE-COUNTRY-UNIVERSE SERIES

def vector_autocorr(ser_source, int_shift):
    ### Defining adding full universe for each date:
    def universe_reindex(iter_group, idx_universe):
        df_iter_result = iter_group.unstack('Date').reindex(idx_universe).sort_index().stack('Date', dropna = False)
        ### Results output:
        return df_iter_result   
    ### Defining adding full date range for each country and date index shifting:
    def date_reindex(iter_group, idx_date_range, num_shift = 0):
        ser_iter_result = iter_group.unstack('Country').reindex(idx_date_range).sort_index().shift(-int_shift).stack('Country', dropna = False).squeeze()
        ### Results output:
        return ser_iter_result   
    ### Defining by date correrlation function:
    def corr_by_date(iter_group):
        num_iter_corr = iter_group['Corr_factor_minus'].corr(iter_group['Corr_factor_plus'])
        ### Results output:
        return num_iter_corr       
    ### Preparing expanded universe for autocorrelation performing:
    idx_date_range = ser_source.index.get_level_values(0).unique()
    idx_universe = ser_source.index.get_level_values(1).unique()
    ser_source_full = ser_source.to_frame().reset_index('Market', drop = True).groupby('Date', group_keys = False).apply(universe_reindex, idx_universe)\
                                .swaplevel().squeeze()
    ### Autocorrelation preparing:
    ser_source_plus = ser_source_full.groupby('Country', group_keys = False).apply(lambda iter_group: iter_group.iloc[1 : ])\
                                     .sort_index(level = ['Date', 'Country'])
    ser_source_minus = ser_source_full.groupby('Country', group_keys = False).apply(lambda iter_group: iter_group.iloc[: -1])\
                                      .sort_index(level = ['Date', 'Country'])
    ### Artificial series combining for indexes synchronization:        
    ser_source_plus_shifted = ser_source_plus.groupby('Country', group_keys = False).apply(date_reindex, idx_date_range, int_shift)
    df_to_corr = pd.concat([ser_source_minus, ser_source_plus_shifted], axis = 1)
    df_to_corr.columns = ['Corr_factor_minus', 'Corr_factor_plus']
    ser_autocorr_vector = df_to_corr.groupby('Date').apply(corr_by_date).shift(int_shift)
    ### Results output:
    return ser_autocorr_vector

In [13]:
# DEFINING MULTI-STEP STANDARTIZATION BY MARKET FOR FULL FACTOR STACK

def single_factor_standartize(ser_factor, arr_truncate, ser_weight = False, reuse_outliers = False, center_result = True, within_market = False, 
                              flag_tha = False, flo_similarity = 5 * (10 ** (-8))):
    ### Local constants:
    dict_tha_pow = {}
    dict_tha_pow['monthly'] = 1
    dict_tha_pow['quarterly'] = 1 / 3
    dict_tha_pow['annual'] = 1 / 12
    ### Weights preparing:
    if isinstance(ser_weight, bool):
        ser_weight = pd.Series(1, index = ser_factor.index)
        ser_weight.name = 'Weight'
    ### Multi-step standartizing:        
    df_factor = ser_factor.to_frame().join(ser_weight, how = 'left')
    df_factor.columns = ['Factor', 'Weight']
    ### Time-horizon adjusted standartization:
    if (flag_tha):
        ### Demeaned by region z-scored vector calculating:       
        ser_stand_z = df_factor.groupby('Date', group_keys = False)\
                               .apply(lambda iter_df: tha_standartize(iter_df['Factor'], arr_truncate, iter_df['Weight'], reuse_outliers, center_result, False))
        ### Results output:
        ser_stand_z.name = ser_factor.name
        ### Autocorrelation vector calculating:
        ser_autocorr_vector = ser_stand_z.groupby('Market').apply(vector_autocorr, 1)
        ser_autocorr_vector.name = 'Autocorr'
        if (flag_tha == 'monthly'):
            ser_autocorr_cum_mean = ser_autocorr_vector.groupby('Market', group_keys = False).expanding().mean()
        else:
            ser_autocorr_cum_mean = ser_autocorr_vector.loc[np.abs(ser_autocorr_vector - 1) > flo_similarity].groupby('Market', group_keys = False).expanding().mean()
        ### THA-coeficcient calculating:
        ser_tha_coeff = ser_autocorr_cum_mean.transform(lambda iter_mean: max(iter_mean, 0.0) ** dict_tha_pow[flag_tha])
        ser_tha_coeff = ser_tha_coeff.transform(lambda iter_mean: 
                                                sum(map(lambda iter_num: geom_weight_single(flo_tha_ratio * iter_mean, 1, iter_num), range(int_tha_length))) / 2)
        ser_tha_coeff = ser_tha_coeff.swaplevel()
        ser_tha_coeff = ser_tha_coeff.unstack('Market').reindex(ser_stand_z.index.levels[0]).stack('Market', dropna = False).sort_index(level = ['Date', 'Market'])        
        ### THA-adjusted z-score calculating:
#        ser_stand_s = (ser_stand_z * ser_tha_coeff)
        ### Artifical filling values for first date of region appearance (not to loose observations):
        ser_stand_s = (ser_stand_z * ser_tha_coeff.fillna(2.0)) ### Result of geometric progression sum with assumption that correlation is equal to 0.55
        ser_stand_s = ser_stand_s[ser_stand_s.index.dropna()].reorder_levels(['Date', 'Country', 'Market']).sort_index()
        ### Standart deviation for THA-adjusted z-score calculating:
        ser_region_std = ser_stand_s.groupby(['Date', 'Market']).std()
        ser_universe_std = ser_stand_s.groupby(['Date']).std()
        ser_universe_std = pd.concat([ser_universe_std], keys = ['Overall'], names = ['Market']).swaplevel()
        ser_std = pd.concat([ser_region_std, ser_universe_std], axis = 0).sort_index()
        ### Results output:
        return (ser_stand_s, ser_stand_z, ser_autocorr_vector, ser_tha_coeff, ser_std)
    ### Simple standartization:    
    else:    
        ser_result = df_factor.groupby('Date', group_keys = False)\
                     .apply(lambda iter_df: ison_standartize(iter_df['Factor'], arr_truncate, iter_df['Weight'], reuse_outliers, center_result, False, within_market))
        ### Results output:
        ser_result.name = ser_factor.name
        return ser_result

In [14]:
### DEFINING EXTRACTION UNIVERSE DATA FROM MS EXCEL SOURCE

def ison_membership_converting(str_path_universe, date_end, bool_daily = False, int_backfill_months = 0):
    ### Defining business-month-end reindexation on country level:
    def country_modify(ser_raw_country, date_end):
        ser_res_country = ser_raw_country.droplevel(0).resample('MS').last().resample('BM').last()
        range_country = pd.date_range(ser_res_country.index[0], date_end, freq = 'BM')
        return ser_res_country.reindex(range_country).ffill()
    ### Markets encoding table:
    dict_markets = {50 : 'DM', 57 : 'EM', 504 : 'FM', 0: np.NaN}     
    ### Loading source file:
    df_raw_universe = pd.read_excel(io = str_path_universe, sheet_name = 0, header = 0, parse_dates = True, index_col = [0, 1],
                                 na_values = ['', '#N/A', '#N/A N/A', '#NA', '-1.#IND', '-1.#QNAN', '-NaN', '-nan', '1.#IND', 
                                             '1.#QNAN', 'N/A', 'NULL', 'NaN', 'n/a', 'nan', 'null'], keep_default_na = False)
    ### Converting source file:
    df_raw_universe.index.names = ['Country', 'Date']
    ser_raw_universe = df_raw_universe['Region']
    ser_raw_universe.fillna(0, inplace = True)
    ser_raw_universe.name = 'Market'
    ### By country reindexation and translation:
    ser_res_universe = ser_raw_universe.groupby('Country').apply(country_modify, date_end)
    ser_res_universe.index.names = ['Country', 'Date']
    ser_res_universe = ser_res_universe.replace(dict_markets).reorder_levels([1, 0]).sort_index() 
    ### Expanding membership for primary regions members by backfilling:
    if int_backfill_months:
        ### List of regions:
        list_region = list(ser_res_universe.dropna().unique())
        ### Initialising of collection of series with backfilled data for each region:
        list_ison_backfill = []
        ### Regions looping:
        for iter_region in list_region:
            ### Defining start of region date:
            date_first_valid = ser_res_universe.loc[ser_res_universe == iter_region].first_valid_index()[0]
            ### Creating dates index to backfilling:
            idx_date_backfill = pd.date_range(end = date_first_valid, periods = int_backfill_months + 1, freq = 'BM')[: -1]
            ### Creating primary countries index to backfilling:            
            idx_region_backfill = ser_res_universe.loc[ser_res_universe == iter_region].loc[date_first_valid, All].index.get_level_values('Country')
            ### Creating full index:
            idx_ison_backfill = pd.MultiIndex.from_product([idx_date_backfill, idx_region_backfill])
            ### Series with backfilled data:
            list_ison_backfill.append(pd.Series(iter_region, index = idx_ison_backfill))
        ### Combination of backfilled series and original ISON data:    
        ser_res_universe = ser_res_universe.combine_first(pd.concat(list_ison_backfill, axis = 0)).sort_index()  
        ser_res_universe.index.names = ['Date', 'Country']
    ### Converting to daily frequency:
    if bool_daily:
        ser_res_universe = ser_res_universe.reset_index('Country').groupby('Country').resample('B').ffill()['Market'].swaplevel().sort_index()    
    ### Results output:
    ser_res_universe.name = 'Market'
    return ser_res_universe

In [15]:
### DATA LOADING AND CONVERTING

ser_reer = pd.read_hdf(str_path_bb_hdf, key = str_key_reer_sourced) ### Real Effective Exchange Rate to use as a source in the Short-Term factor
int_eer_fill_limit = 66
ser_reer_source = ser_reer.droplevel('Source').unstack('Country').reindex(idx_source_date_range).ffill(limit = int_eer_fill_limit).stack('Country').sort_index()
ser_reer_source.index.names = ['Date', 'Country']

ser_ison = ison_membership_converting(str_path_universe, datetime.strptime(str_measure_date_end, '%Y-%m-%d')) ### ISON universe, end-of-business-months vector

In [16]:
### MAIN SCRIPT: STANDALONE REER FACTOR CALCULATING

### Factor matrix creating:
ser_reer_factor = pd.Series(index = pd.MultiIndex.from_product([idx_factor_date_range, ser_ison.index.get_level_values(1).unique()])).sort_index()
ser_reer_factor.index.set_names(['Date', 'Country'], inplace = True)                
### Source performing:
ser_reer_delta = ser_reer_source.groupby('Country').diff() / ser_reer_source.groupby('Country').shift()   
ser_reer_delta = ser_reer_delta.replace([np.inf, -np.inf], np.NaN)
### Momentum parameters:
int_mom_hl = 520 ### Without rounding here
int_mom_win = 1300
int_mom_min = 520
### Weights array:
list_weight = list(map(lambda iter_num: exp_weight_single(int_mom_hl, iter_num), range(int_mom_win)))[::-1]
### Momentum factor calculation:
ser_reer_factor = ser_reer_factor.groupby('Country').transform(rolling_cond_weighted_mean, ser_reer_delta, int_mom_win, int_mom_min, list_weight, False)
### Factor ISONing:
ser_reer_factor = ser_reer_factor.to_frame().join(ser_ison, how = 'left').set_index('Market', append = True).squeeze()
ser_reer_factor.name = 'Factor'               
### Regions clearing:
ser_reer_factor = ser_reer_factor.loc[idx_factor_date_range, All, list_ison]
### Countries filtering:
ser_reer_factor = ser_reer_factor.drop(list_countries_to_exclude, level = 'Country') 
### Standalone factor standartizing:
#ser_reer_factor_std = -single_factor_standartize(ser_reer_factor, list_truncate, within_market = bool_within_market)
#ser_reer_factor_std.name = 'Factor' 
#tup_reer_factor_tha = single_factor_standartize(ser_reer_factor_std, list_truncate, within_market = bool_within_market, flag_tha = 'monthly')
tup_reer_factor_tha = single_factor_standartize(-ser_reer_factor, list_truncate, within_market = bool_within_market, flag_tha = 'monthly')

In [20]:
### TEMP

#ser_reer_tha = tup_reer_factor_tha[0]
#ser_reer_tha
#ser_reer_tha.to_excel(str_reer_tha_xlsx, merge_cells = False)
ser_reer_factor.to_excel(str_reer_raw_xlsx, merge_cells = False)

Date        Country  Market  Market
1994-01-31  AT       DM      DM        0.000056
            AU       DM      DM             NaN
            BE       DM      DM       -0.000009
            CA       DM      DM             NaN
            CH       DM      DM             NaN
                                         ...   
2020-08-31  UG       FM      FM        0.000069
            US       DM      DM        0.000025
            VN       FM      FM        0.000047
            ZA       EM      EM       -0.000126
            ZM       FM      FM       -0.000382
Name: Factor, Length: 19621, dtype: float64